In [75]:
import pandas as pd
import numpy as np

In [76]:
house = pd.read_csv('data/raw/pp-complete.csv', names= [
    "transaction_id",      # 1
    "price",               # 2
    "date_of_transfer",    # 3
    "postcode",            # 4
    "property_type",       # 5
    "new_build",           # 6
    "tenure",              # 7
    "primary_address",     # 8 (PAON)
    "secondary_address",   # 9 (SAON)
    "street",              # 10
    "locality",            # 11
    "town_city",           # 12
    "district",            # 13
    "county",              # 14
    "ppd_category_type",   # 15
    "record_status"        # 16 (only in monthly updates)
]
                    )
house.head()


,transaction_id,price,date_of_transfer,postcode,property_type,new_build,tenure,primary_address,secondary_address,street,locality,town_city,district,county,ppd_category_type,record_status
0,{F887F88E-7D15-4415-804E-52EAC2F10958},70000,1995-07-07 00:00,MK15 9HP,D,N,F,31,NaN,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES,A,A
1,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03 00:00,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13 00:00,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
3,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28 00:00,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
4,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28 00:00,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A


In [77]:
# Load the ONS postcode data (takes a few seconds)
onspd = pd.read_csv("data/raw/ONSPD_MAY_2025_UK.csv", encoding='latin1', dtype={'pcds': str})

# Clean both for safe merging
house['postcode'] = house['postcode'].str.strip().str.upper()
onspd['pcds'] = onspd['pcds'].str.strip().str.upper()

# Merge to bring in latitude and longitude
house = house.merge(onspd[['pcds', 'lat', 'long']], left_on='postcode', right_on='pcds', how='left')

# Drop extra postcode column if needed
house.drop(columns='pcds', inplace=True)

# Now merged has lat/lon for most postcodes
print(house[['postcode', 'lat', 'long']].head())


/var/folders/qb/p7j426sx0lvb8p_g2qrd7z400000gn/T/ipykernel_64662/2075820334.py:2: DtypeWarning: Columns (18,31,39,44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  onspd = pd.read_csv("ONSPD_MAY_2025_UK.csv", encoding='latin1', dtype={'pcds': str})


   postcode        lat      long
0  MK15 9HP  52.065059 -0.720241
1   SR6 0AQ  54.912547 -1.380991
2   CO6 1SQ  51.877253  0.692923
3   B90 4TG  52.389317 -1.801110
4   DY5 1SA  52.475009 -2.109250


In [78]:
# Filtering down location city Bristol
house =  house[house["town_city"] =="BRISTOL"]

In [79]:
house.head()

,transaction_id,price,date_of_transfer,postcode,property_type,new_build,tenure,primary_address,secondary_address,street,locality,town_city,district,county,ppd_category_type,record_status,lat,long
20,{3A947F9C-D5C3-4178-8382-566E952D20A1},47000,1995-10-30 00:00,BS15 9UR,S,N,F,9,NaN,AMBLE CLOSE,BRISTOL,BRISTOL,KINGSWOOD,AVON,A,A,51.458778,-2.495583
28,{CB70FDBB-49CC-412F-911C-5A069B724EE2},110000,1995-12-21 00:00,BS15 3HH,D,N,F,148,NaN,HIGH STREET,HANHAM,BRISTOL,KINGSWOOD,AVON,A,A,51.446917,-2.510347
33,{C2950A84-E385-46C3-AAC3-4F498A1E6857},265000,1995-02-24 00:00,BS9 3EB,D,Y,F,32,NaN,HOLMWOOD GARDENS,BRISTOL,BRISTOL,BRISTOL,AVON,A,A,51.496327,-2.611693
231,{EE33D787-F547-419F-93F7-566F908A7F59},113500,1995-09-06 00:00,BS36 2BB,D,N,F,12,NaN,CHURCH CLOSE,FRAMPTON COTTERELL,BRISTOL,NORTHAVON,AVON,A,A,51.535008,-2.478445
256,{68BA9F61-4110-4D7D-8883-52ED14F3A99C},170000,1995-10-16 00:00,BS40 9XD,D,N,F,THE WHITE HOUSE,NaN,POTTERS HILL,FELTON,BRISTOL,WOODSPRING,AVON,A,A,51.392278,-2.696426


In [80]:
len(house)

464387

In [81]:
# check for missing values
house.isnull().sum()

transaction_id            0
price                     0
date_of_transfer          0
postcode                665
property_type             0
new_build                 0
tenure                    0
primary_address           3
secondary_address    396820
street                  977
locality             126382
town_city                 0
district                  0
county                    0
ppd_category_type         0
record_status             0
lat                     680
long                    680
dtype: int64

In [82]:
# Drop secondary address (lots of missing values)
house.drop("secondary_address", axis = 1, inplace = True)
house.head()

,transaction_id,price,date_of_transfer,postcode,property_type,new_build,tenure,primary_address,street,locality,town_city,district,county,ppd_category_type,record_status,lat,long
20,{3A947F9C-D5C3-4178-8382-566E952D20A1},47000,1995-10-30 00:00,BS15 9UR,S,N,F,9,AMBLE CLOSE,BRISTOL,BRISTOL,KINGSWOOD,AVON,A,A,51.458778,-2.495583
28,{CB70FDBB-49CC-412F-911C-5A069B724EE2},110000,1995-12-21 00:00,BS15 3HH,D,N,F,148,HIGH STREET,HANHAM,BRISTOL,KINGSWOOD,AVON,A,A,51.446917,-2.510347
33,{C2950A84-E385-46C3-AAC3-4F498A1E6857},265000,1995-02-24 00:00,BS9 3EB,D,Y,F,32,HOLMWOOD GARDENS,BRISTOL,BRISTOL,BRISTOL,AVON,A,A,51.496327,-2.611693
231,{EE33D787-F547-419F-93F7-566F908A7F59},113500,1995-09-06 00:00,BS36 2BB,D,N,F,12,CHURCH CLOSE,FRAMPTON COTTERELL,BRISTOL,NORTHAVON,AVON,A,A,51.535008,-2.478445
256,{68BA9F61-4110-4D7D-8883-52ED14F3A99C},170000,1995-10-16 00:00,BS40 9XD,D,N,F,THE WHITE HOUSE,POTTERS HILL,FELTON,BRISTOL,WOODSPRING,AVON,A,A,51.392278,-2.696426


In [83]:
# date
house["Year"] = house["date_of_transfer"].apply(lambda x: pd.to_datetime(x).year)
house["Month"] = house["date_of_transfer"].apply(lambda x: pd.to_datetime(x).month)
house["Day"] = house["date_of_transfer"].apply(lambda x: pd.to_datetime(x).day)
house.drop("date_of_transfer", axis = 1, inplace = True)

In [91]:
house.head()

,transaction_id,price,postcode,property_type,new_build,tenure,primary_address,street,locality,town_city,district,county,ppd_category_type,record_status,lat,long,Year,Month,Day
20,{3A947F9C-D5C3-4178-8382-566E952D20A1},47000,BS15 9UR,S,N,F,9,AMBLE CLOSE,BRISTOL,BRISTOL,KINGSWOOD,AVON,A,A,51.458778,-2.495583,1995,10,30
28,{CB70FDBB-49CC-412F-911C-5A069B724EE2},110000,BS15 3HH,D,N,F,148,HIGH STREET,HANHAM,BRISTOL,KINGSWOOD,AVON,A,A,51.446917,-2.510347,1995,12,21
33,{C2950A84-E385-46C3-AAC3-4F498A1E6857},265000,BS9 3EB,D,Y,F,32,HOLMWOOD GARDENS,BRISTOL,BRISTOL,BRISTOL,AVON,A,A,51.496327,-2.611693,1995,2,24
231,{EE33D787-F547-419F-93F7-566F908A7F59},113500,BS36 2BB,D,N,F,12,CHURCH CLOSE,FRAMPTON COTTERELL,BRISTOL,NORTHAVON,AVON,A,A,51.535008,-2.478445,1995,9,6
256,{68BA9F61-4110-4D7D-8883-52ED14F3A99C},170000,BS40 9XD,D,N,F,THE WHITE HOUSE,POTTERS HILL,FELTON,BRISTOL,WOODSPRING,AVON,A,A,51.392278,-2.696426,1995,10,16


In [92]:
# shape of dataset
house.shape

(464387, 19)

In [93]:
#data type
house.dtypes

transaction_id        object
price                  int64
postcode              object
property_type         object
new_build             object
tenure                object
primary_address       object
street                object
locality              object
town_city             object
district              object
county                object
ppd_category_type     object
record_status         object
lat                  float64
long                 float64
Year                   int64
Month                  int64
Day                    int64
dtype: object

In [94]:
house.describe()

,price,lat,long,Year,Month,Day
count,4.643870e+05,463707.000000,463707.000000,464387.000000,464387.000000,464387.000000
mean,2.335767e+05,51.470107,-2.573913,2009.302435,6.771096,17.215189
std,6.781292e+05,0.048400,0.085161,8.717707,3.350346,9.046965
min,1.000000e+02,51.274544,-2.841747,1995.000000,1.000000,1.000000
25%,1.100000e+05,51.441837,-2.609552,2002.000000,4.000000,10.000000
50%,1.780000e+05,51.468421,-2.568539,2009.000000,7.000000,18.000000
75%,2.780000e+05,51.494716,-2.518002,2017.000000,10.000000,25.000000
max,1.661053e+08,51.663267,-2.335964,2025.000000,12.000000,31.000000


In [112]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor      # swap any model you like
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

# -------------------------------------------------------------------
# 0️⃣  Split   -------------------------------------------------------
x = house[["property_type", "new_build", "tenure",
           "Year", "Month", "Day", "lat", "long"]]
y = house["price"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, random_state=18
)

# -------------------------------------------------------------------
# 1️⃣  Define column groups -----------------------------------------
cat_cols = ["property_type", "new_build", "tenure"]
num_cols = ["Year", "Month", "Day", "lat", "long"]

# -------------------------------------------------------------------
# 2️⃣  Build preprocessing pipeline ---------------------------------
preprocess = ColumnTransformer(
    transformers=[
        # Categorical: impute (most frequent) ➜ one-hot
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
        # Numeric: impute (median) ➜ pass through
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median"))
        ]), num_cols)
    ]
)

# -------------------------------------------------------------------
# 3️⃣  Full model pipeline ------------------------------------------

model = Pipeline([
    ("prep", preprocess),
    ("catb", CatBoostRegressor(
        iterations=600,
        learning_rate=0.05,
        depth=8,
        loss_function="MAE",
        random_state=18,
        verbose=0          # suppress training log
    ))
])

# -------------------------------------------------------------------
# 4️⃣  Fit & evaluate ------------------------------------------------
model.fit(x_train, y_train)
pred = model.predict(x_test)
mae  = mean_absolute_error(y_test, pred)
print(f"Mean Absolute Error: {mae:,.0f}")


Mean Absolute Error: 62,031
